In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [37]:
skt_url='https://lol.gamepedia.com/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Team&MHG%5Blimit%5D=130&MHG%5Bteam%5D=SK%20Telecom%20T1&MHG%5Btextonly%5D=Yes&MHG%5Bspl%5D=yes&pfRunQueryFormName=MatchHistoryGame'
damwon_url='https://lol.gamepedia.com/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Team&MHG%5Blimit%5D=120&MHG%5Bteam%5D=DAMWON%20Gaming&MHG%5Btextonly%5D=Yes&MHG%5Bspl%5D=yes&pfRunQueryFormName=MatchHistoryGame'
griffin_url='https://lol.gamepedia.com/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Team&MHG%5Blimit%5D=100&MHG%5Bteam%5D=Griffin&MHG%5Btextonly%5D=Yes&MHG%5Bspl%5D=yes&pfRunQueryFormName=MatchHistoryGame'

In [3]:
def content_grab(url):
    r=requests.get(url)
    soup=BeautifulSoup(r.content, "html")
    body_html=soup.find("div", {"class":"mw-body"})
    table_html=body_html.find_all("td")
    parse_html=[i.text.strip() for i in table_html]
    array_html=np.asarray(list(parse_html[:(len(parse_html)-22)])).reshape((-1,25))
    column_name= ['Date', 'Tournament', 'P', 'W/L', 'Side', 'Vs', 'Bans', 
                  'BansVs', 'Picks', 'PicksVs', 'Players', 'Len', 'Team_G', 'Team_K', 'Team_T', 'Team_D', 'Team_B', 
                  'VS_G', 'VS_K', 'VS_T', 'VS_D', 'VS_B', 'SB', 'MH', 'VOD']
    raw_df = pd.DataFrame(array_html, columns=column_name)
    df=raw_df.copy()
    df['datetime']=pd.to_datetime(df['Date'])
    df.drop(['Date'], axis=1, inplace=True)
    df=df[df['datetime']>='2019-01-01']
    df.drop(['Vs','SB', 'MH', 'VOD'], axis=1, inplace=True)
    return df

In [38]:
df=content_grab(griffin_url)

In [39]:
df.columns

Index(['Tournament', 'P', 'W/L', 'Side', 'Bans', 'BansVs', 'Picks', 'PicksVs', 'Players', 'Len', 'Team_G', 'Team_K', 'Team_T', 'Team_D', 'Team_B', 'VS_G', 'VS_K', 'VS_T', 'VS_D', 'VS_B', 'datetime'], dtype='object')

In [40]:
def count_champs(df, col):
    return pd.Series(df[col].str.cat(sep=',').split(',')).value_counts().head(10)

In [41]:
def champ_count_summary(df):
    Bans=count_champs(df,'Bans').reset_index()
    BansVs=count_champs(df,'BansVs').reset_index()
    Picks=count_champs(df,'Picks').reset_index()
    PicksVs=count_champs(df,'PicksVs').reset_index()
    summary_df=pd.concat([Bans, BansVs, Picks, PicksVs],axis=1)
    summary_df.columns=['Bans', 'Count', 'BansVs', 'Count', 'Picks', 'Count', 'PicksVs', 'Count']
    return summary_df

In [42]:
champ_count_summary(df)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Kalista,25,Yuumi,37,Aatrox,24,Sejuani,19
1,Karma,21,Akali,27,Jayce,22,Braum,18
2,Aatrox,19,Irelia,26,Ezreal,20,Jayce,18
3,LeBlanc,18,Jayce,24,Kai'Sa,19,Aatrox,17
4,Irelia,17,Galio,20,Sejuani,17,Ezreal,16
5,Varus,17,Sejuani,18,Tahm Kench,16,Lucian,15
6,Lucian,15,Ezreal,18,Akali,15,Tahm Kench,14
7,Neeko,14,Olaf,17,Galio,14,Lissandra,14
8,Thresh,14,Aatrox,15,Lucian,14,Jarvan IV,13
9,Skarner,13,Karma,15,Olaf,13,Lee Sin,13


In [43]:
df_loss=df[df['W/L']=='Loss']

In [44]:
champ_count_summary(df_loss)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Varus,10,Yuumi,16,Aatrox,9,Braum,9
1,Kalista,9,Jayce,9,Kai'Sa,8,Ezreal,8
2,Neeko,9,Irelia,9,Sylas,7,Sejuani,6
3,LeBlanc,8,Lux,8,Olaf,5,Lee Sin,6
4,Jayce,7,Karma,6,Ezreal,5,Galio,6
5,Aatrox,7,Akali,6,Sejuani,5,Jayce,6
6,Tahm Kench,5,Tahm Kench,5,Galio,5,Akali,6
7,Sylas,5,Kalista,5,Tahm Kench,5,Rakan,5
8,Karma,5,Galio,5,Lissandra,4,Aatrox,5
9,Skarner,4,Kennen,5,Jayce,4,Tahm Kench,4


In [45]:
df_loss_blue=df[(df['W/L']=='Loss') & (df['Side']=='Blue')]

In [46]:
champ_count_summary(df_loss_blue)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Neeko,4,Yuumi,8,Sylas,5,Sejuani,4
1,Varus,4,Irelia,5,Aatrox,4,Braum,4
2,Tahm Kench,3,Lux,4,Kai'Sa,3,Tahm Kench,4
3,Karma,3,Jayce,4,Jarvan IV,3,Corki,3
4,Jayce,3,Kalista,4,Akali,3,Ryze,3
5,Skarner,3,Akali,4,Tahm Kench,3,Ezreal,3
6,Morgana,3,Karma,3,Kennen,2,Aatrox,3
7,Rek'Sai,3,Galio,3,Corki,2,Jarvan IV,3
8,Aatrox,3,Ezreal,3,Nautilus,2,Rakan,2
9,LeBlanc,3,Trundle,2,Karma,2,Jayce,2


In [47]:
df_loss_red=df[(df['W/L']=='Loss') & (df['Side']=='Red')]

In [48]:
champ_count_summary(df_loss_red)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Kalista,7,Yuumi,8,Aatrox,5,Galio,6
1,Varus,6,Jayce,5,Kai'Sa,5,Akali,6
2,LeBlanc,5,Olaf,4,Ezreal,4,Lee Sin,5
3,Neeko,5,Lux,4,Sejuani,3,Ezreal,5
4,Jayce,4,Irelia,4,Olaf,3,Braum,5
5,Aatrox,4,Tahm Kench,4,Lissandra,3,Lucian,4
6,Lucian,3,Karma,3,Galio,3,Jayce,4
7,Alistar,3,Yorick,3,Ryze,3,Rakan,3
8,Sylas,3,Nocturne,3,Jayce,2,Aatrox,2
9,Renekton,2,Kennen,3,Shen,2,Lissandra,2


In [49]:
df_loss_red.shape[0]

15

In [50]:
df_loss_blue.shape[0]

13

In [51]:
df_win=df[df['W/L']=='Win']

In [52]:
champ_count_summary(df_win)

,Bans,Count,BansVs,Count,Picks,Count,PicksVs,Count
0,Karma,16,Akali,21,Jayce,18,Sejuani,13
1,Kalista,16,Yuumi,21,Aatrox,15,Jayce,12
2,Irelia,14,Irelia,17,Ezreal,15,Aatrox,12
3,Thresh,13,Jayce,15,Sejuani,12,Lucian,11
4,Aatrox,12,Galio,15,Lucian,12,Kai'Sa,11
5,Lucian,12,Ezreal,14,Akali,12,Lissandra,11
6,Rakan,10,Sejuani,14,Kai'Sa,11,Urgot,11
7,LeBlanc,10,Olaf,13,Tahm Kench,11,Tahm Kench,10
8,Taliyah,9,Aatrox,12,Sion,10,Alistar,9
9,Skarner,9,Sylas,11,Urgot,9,Gragas,9
